Este notebook apunta a realizar los análisis de datos necesarios para responder a la pregunta siguiente: 

- ¿Cuáles son las caractéristicas sociales de las personas que hablan sobre el cambio climático en la prensa en terminos de "edad", "genero", "profesion" y "popularidad"? 

- ¿Cómo evolucionan estas características en el tiempo? 

- ¿Se puede observar tendencias distintas en los paises siguientes: Costa Rica, España, Chile

ver más detalles en el documento borrador: https://docs.google.com/document/d/1LyIBSMz_Suhy7esHSQt49P56asrhoeCb18qbpOggCY8/edit?usp=sharing

# 1- Construcción del corpus de noticias

In [4]:
import pandas as pd
from pandasql import sqldf

In [2]:
file_corpus_chile="cambio_climatico_chile_2018-01-01_2021-12-31.csv"

In [3]:
corpus_chile=pd.read_csv(file_corpus_chile)

In [4]:
corpus_chile

,Unnamed: 0,id_news,country,media_outlet,url,title,text,date,search
0,0,13760981.0,chile,emol,https://www.emol.com/noticias/Nacional/2019/08...,"Piñera: ""Quiero que seamos capaces de anticipa...",En medio de la celebración el Día del Campesin...,2019-08-01,cambio climático
1,1,5213042.0,chile,elciudadano,https://www.elciudadano.com/medio-ambiente/gra...,Gran parte de la crisis humanitaria mundial se...,El cambio climático fue responsable de la mayo...,2019-02-21,cambio climático
2,2,4141229.0,chile,latercera,https://www.latercera.com/politica/noticia/gui...,"""¿Cuál es la posición oficial en cambio climát...",El senador independiente y excandidato preside...,2019-09-26,cambio climático
3,3,5182610.0,chile,elciudadano,https://www.elciudadano.com/medio-ambiente/esp...,España fija la lucha contra el calentamiento g...,El plan del nuevo Ejecutivo español es remitir...,2020-01-21,cambio climático
4,4,6417672.0,chile,latercera,https://www.latercera.com/que-pasa/noticia/inc...,"Incentivos para donaciones, ciencia y rol del ...","""Chile es un país altamente vulnerable a los e...",2020-01-10,cambio climático
...,...,...,...,...,...,...,...,...,...
4488,4705,13355152.0,chile,emol,https://www.emol.com/noticias/Nacional/2021/03...,Streaming y entrevistas: La programación de es...,"#nota_tabla_emol .tablaennoticia tr td,#nota_...",2021-03-21,cambio climático
4489,4707,6818889.0,chile,latercera,https://www.latercera.com/masdeco/talentos-201...,Talentos 2019 parte I,SUSTENTABILIDAD Ecolover: Sustentabilidad y re...,2019-11-14,cambio climático
4490,4708,6818961.0,chile,latercera,https://www.latercera.com/masdeco/talentos-201...,Talentos 2019 parte II,ARTE Delight lab: Artistas y activistas lumíni...,2019-11-14,cambio climático
4491,4709,4032370.0,chile,elciudadano,https://www.elciudadano.com/mundo/cumbre-g-20-...,Cumbre G-20: Bolsonaro tensiona el Mercosur en...,"En menos de dos semanas, los presidentes de Es...",2018-11-19,cambio climático


- ¿Cuáles son los medios y su número de noticias?

In [5]:
query="""SELECT media_outlet,count(*) as noticias, min(date),max(date)
FROM corpus_chile GROUP BY media_outlet ORDER BY noticias DESC;"""
result=sqldf(query)
result

,media_outlet,noticias,min(date),max(date)
0,latercera,1435,2018-01-18,2021-01-27
1,emol,1054,2018-01-03,2021-04-23
2,elciudadano,686,2018-01-02,2021-01-12
3,biobiochile,565,2018-01-01,2020-12-26
4,ahoranoticiasmega,465,2018-01-05,2021-01-18
5,elmostrador,161,2018-01-01,2021-01-25
6,horas24,127,2018-01-25,2020-11-28


- ¿Cómo se distribuyen las noticias en el tiempo?

In [6]:
#Contar el número de documentos por año

query="""SELECT strftime('%Y', date) as year,count(*) as documents
FROM corpus_chile GROUP BY year;"""
result=sqldf(query)
result

,year,documents
0,2018,920
1,2019,2416
2,2020,1038
3,2021,119


# 2. Detección de las entidades de tipo Personas

In [7]:
from collections import Counter
import spacy
nlp = spacy.load("es_core_news_lg") #cambiar por "es_core_news_lg" para el análisis final

In [8]:
df_persons = pd.DataFrame(columns=["date", "name"])

for index, row in corpus_chile.iterrows():
    print(index)
    text = str(row["text"])
    date = row["date"]
    person_in_text = []
    
    if(len(text)>0):
        doc = nlp(text)
        
        for ent in doc.ents:
            if (ent.label_ == "PER"):
                person = ent.text
                if (' ' in person) == True:
                    if person not in person_in_text:
                        df_persons = df_persons.append({"date": date,"name":  person}, ignore_index=True)
                        person_in_text.append(person)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062


3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702


In [9]:
df_persons

,date,name
0,2019-08-01,Sebastián Piñera
1,2019-08-01,Antonio Walker
2,2019-02-21,El Niño
3,2019-02-21,Sven Harmeling
4,2019-02-21,Asad Rehman
...,...,...
18584,2019-08-31,Francis Lawrence
18585,2019-08-31,Keanu Reeves
18586,2019-08-31,John Constantine
18587,2019-08-31,Will Smith


In [10]:
query="""SELECT strftime('%Y', date) as year,name, count(*) as freq
FROM df_persons GROUP BY year, name ORDER BY count(*) DESC;"""
df_persons_by_year=sqldf(query)
df_persons_by_year

,year,name,freq
0,2019,Sebastián Piñera,371
1,2019,Greta Thunberg,292
2,2019,Donald Trump,226
3,2019,Carolina Schmidt,198
4,2018,Donald Trump,155
...,...,...,...
10214,2021,Yuval Harary,1
10215,2021,la Niña,1
10216,2021,Álvaro Ortiz,1
10217,2021,Álvaro Quintanilla,1


# 3. Extracción de metadatos sobre cada persona (género, popularidad, edad, profesión)

- GENERO

In [11]:
import unidecode

DICT_FIRSTNAMES="names_gender.csv"

gen_nam=pd.read_csv(DICT_FIRSTNAMES)

In [12]:
def get_gender(named_entity, language):
    #We consider the first token  of the named entity as the firstname
    firstname=named_entity.split(' ', maxsplit=1)[0] 
    firstname_upper= firstname.upper()
    firstname_unidecode_upper=unidecode.unidecode(firstname).upper()

    gender_df = gen_nam[(gen_nam.firstname==firstname) | (gen_nam.firstname==firstname_upper) | (gen_nam.firstname==firstname_unidecode_upper)]
    male_df = gender_df[gender_df.gender=='M']
    female_df = gender_df[gender_df.gender=='F']
    if(len(gender_df)!=0):
        if(len(female_df)/len(gender_df) >= 0.6  ):
            return female_df.values[0][2]
        if(len(male_df)/len(gender_df) >= 0.6 ):
            return male_df.values[0][2]
        return 'N'    
    return None

In [13]:
get_gender("Jamal Khashoggi","Spanish")

'M'

In [14]:
df_persons_by_year["gender"] = ""


In [15]:
df_persons_by_year

,year,name,freq,gender
0,2019,Sebastián Piñera,371,
1,2019,Greta Thunberg,292,
2,2019,Donald Trump,226,
3,2019,Carolina Schmidt,198,
4,2018,Donald Trump,155,
...,...,...,...,...
10214,2021,Yuval Harary,1,
10215,2021,la Niña,1,
10216,2021,Álvaro Ortiz,1,
10217,2021,Álvaro Quintanilla,1,


In [16]:
for index, row in df_persons_by_year.iterrows():
    print(index)
    gender=get_gender(row["name"],"Spanish")
    df_persons_by_year.loc[index,['gender']] = gender

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061


3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704


5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344


6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987


8432
8433
8434
8435
8436
8437
8438
8439
8440
8441
8442
8443
8444
8445
8446
8447
8448
8449
8450
8451
8452
8453
8454
8455
8456
8457
8458
8459
8460
8461
8462
8463
8464
8465
8466
8467
8468
8469
8470
8471
8472
8473
8474
8475
8476
8477
8478
8479
8480
8481
8482
8483
8484
8485
8486
8487
8488
8489
8490
8491
8492
8493
8494
8495
8496
8497
8498
8499
8500
8501
8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612
8613
8614
8615
8616
8617
8618
8619
8620
8621
8622
8623
8624
8625
8626
8627
8628
8629
8630
8631


10062
10063
10064
10065
10066
10067
10068
10069
10070
10071
10072
10073
10074
10075
10076
10077
10078
10079
10080
10081
10082
10083
10084
10085
10086
10087
10088
10089
10090
10091
10092
10093
10094
10095
10096
10097
10098
10099
10100
10101
10102
10103
10104
10105
10106
10107
10108
10109
10110
10111
10112
10113
10114
10115
10116
10117
10118
10119
10120
10121
10122
10123
10124
10125
10126
10127
10128
10129
10130
10131
10132
10133
10134
10135
10136
10137
10138
10139
10140
10141
10142
10143
10144
10145
10146
10147
10148
10149
10150
10151
10152
10153
10154
10155
10156
10157
10158
10159
10160
10161
10162
10163
10164
10165
10166
10167
10168
10169
10170
10171
10172
10173
10174
10175
10176
10177
10178
10179
10180
10181
10182
10183
10184
10185
10186
10187
10188
10189
10190
10191
10192
10193
10194
10195
10196
10197
10198
10199
10200
10201
10202
10203
10204
10205
10206
10207
10208
10209
10210
10211
10212
10213
10214
10215
10216
10217
10218


In [17]:
df_persons_by_year

,year,name,freq,gender
0,2019,Sebastián Piñera,371,M
1,2019,Greta Thunberg,292,F
2,2019,Donald Trump,226,M
3,2019,Carolina Schmidt,198,F
4,2018,Donald Trump,155,M
...,...,...,...,...
10214,2021,Yuval Harary,1,M
10215,2021,la Niña,1,None
10216,2021,Álvaro Ortiz,1,M
10217,2021,Álvaro Quintanilla,1,M


In [18]:
query="""SELECT year, gender, sum(freq), count(*) as nb_persons
FROM df_persons_by_year GROUP BY year, gender ORDER BY year, gender;"""
result=sqldf(query)
result

,year,gender,sum(freq),nb_persons
0,2018,None,368,297
1,2018,F,670,439
2,2018,M,2194,1310
3,2018,N,21,13
4,2019,None,1189,759
5,2019,F,2406,1009
6,2019,M,6267,2880
7,2019,N,40,30
8,2020,None,429,362
9,2020,F,1057,662


- POPULARIDAD

In [19]:
import statistics
import pageviewapi

In [20]:
def get_popularity(named_entity, date):
    popularity_en=calculate_popularity(named_entity, date,"en")
    popularity_es=calculate_popularity(named_entity, date,"es")
    popularity_fr=calculate_popularity(named_entity, date,"fr")
    popularity_it=calculate_popularity(named_entity, date,"it")
    popularity_pt=calculate_popularity(named_entity, date,"pt")
    return {"popularity_en":popularity_en,"popularity_es":popularity_es,"popularity_fr":popularity_fr,"popularity_it":popularity_it,"popularity_pt":popularity_pt}

In [21]:
import unidecode
from datetime import timedelta, datetime

def calculate_popularity(named_entity, date, language):
    start_date = (date - timedelta(days=365)).strftime('%Y%m%d')
    end_date = (date).strftime('%Y%m%d')
    try: 
        result=pageviewapi.per_article(f'{language}.wikipedia', named_entity, start_date , end_date,
                        access='all-access', agent='all-agents', granularity='daily')    
    except Exception as error:
        return 0
    views = []
    for item in result.items():
        for article in item[1]:
            views.append(article['views'])
    return int(statistics.mean(views))

In [22]:
get_popularity("Gavin Newsom",datetime.now())

{'popularity_en': 8696,
 'popularity_es': 107,
 'popularity_fr': 91,
 'popularity_it': 51,
 'popularity_pt': 45}

In [23]:
df_persons_by_year["popularity_en"] = ""
df_persons_by_year["popularity_es"] = ""
df_persons_by_year["popularity_fr"] = ""
df_persons_by_year["popularity_it"] = ""
df_persons_by_year["popularity_pt"] = ""

In [24]:
 df_persons_by_year

,year,name,freq,gender,popularity_en,popularity_es,popularity_fr,popularity_it,popularity_pt
0,2019,Sebastián Piñera,371,M,,,,,
1,2019,Greta Thunberg,292,F,,,,,
2,2019,Donald Trump,226,M,,,,,
3,2019,Carolina Schmidt,198,F,,,,,
4,2018,Donald Trump,155,M,,,,,
...,...,...,...,...,...,...,...,...,...
10214,2021,Yuval Harary,1,M,,,,,
10215,2021,la Niña,1,None,,,,,
10216,2021,Álvaro Ortiz,1,M,,,,,
10217,2021,Álvaro Quintanilla,1,M,,,,,


- Obtener textos desde Wikipedia, Edad & Profesión

In [25]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from transformers import pipeline

In [26]:
ES_MODEL_LANGUAGE="mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"
EN_MODEL_LANGUAGE="distilbert-base-cased-distilled-squad"

In [27]:
tokenizer_es_language = AutoTokenizer.from_pretrained(ES_MODEL_LANGUAGE)
model_es_language = AutoModelForQuestionAnswering.from_pretrained(ES_MODEL_LANGUAGE)

tokenizer_en_language = AutoTokenizer.from_pretrained(EN_MODEL_LANGUAGE)
model_en_language = AutoModelForQuestionAnswering.from_pretrained(EN_MODEL_LANGUAGE)

In [28]:
q_a_es = pipeline("question-answering", model=model_es_language, tokenizer=tokenizer_es_language)
q_a_en = pipeline("question-answering", model=model_en_language, tokenizer=tokenizer_en_language)

In [29]:
df_persons_by_year["wikipedia"] = ""
df_persons_by_year["birth"] = ""
df_persons_by_year["profession"] = ""

In [30]:
import wikipedia

In [32]:
df_persons_by_year.to_csv("chile_persons_cambioclimatico.csv")

In [34]:
df_persons_by_year = pd.read_csv("chile_persons_cambioclimatico.csv")

In [37]:
for index, row in df_persons_by_year.iterrows():
        print(index)
        name=row["name"]
        print(name)
        year=row["year"]
        
        ##Popularity
        date = datetime(int(year), 12, 31, 0, 0, 0, 0)
        popularities=get_popularity(name,date)
        print(popularities)
        df_persons_by_year.loc[index,['popularity_en']] = popularities["popularity_en"]
        df_persons_by_year.loc[index,['popularity_es']] = popularities["popularity_es"]
        df_persons_by_year.loc[index,['popularity_fr']] = popularities["popularity_fr"]
        df_persons_by_year.loc[index,['popularity_it']] = popularities["popularity_it"]
        df_persons_by_year.loc[index,['popularity_pt']] = popularities["popularity_pt"]
        
        ##Wikipedia
        lang="en"
        wikipedia.set_lang("en")
        try:
            text = wikipedia.summary(name, sentences=3)
        except:
            wikipedia.set_lang("es")
            lang="es"
            try:
                text = wikipedia.summary(name, sentences=3)
            except:
                print(name+" no está en Wikipedia.")
                
        df_persons_by_year.loc[index,['wikipedia']] = text
        
        ##Edad & Profesión
        if lang == "en":
            result=q_a_en(question="What year was she or he born?", context=text)
            df_persons_by_year.loc[index,['birth']] = result["answer"]
            result=q_a_en(question="What is her or his profession?", context=text)
            df_persons_by_year.loc[index,['profession']] = result["answer"]
        else:
            result = q_a_es(question="¿En qué año nació el o ella?", context=text)
            df_persons_by_year.loc[index,['birth']] = result["answer"]
            result = q_a_es(question="¿Cuál es su profesión?", context=text)
            df_persons_by_year.loc[index,['profession']] = result["answer"]

0
Sebastián Piñera
{'popularity_en': 951, 'popularity_es': 2052, 'popularity_fr': 172, 'popularity_it': 157, 'popularity_pt': 266}
1
Greta Thunberg
{'popularity_en': 22508, 'popularity_es': 5219, 'popularity_fr': 4404, 'popularity_it': 5597, 'popularity_pt': 1768}
2
Donald Trump
{'popularity_en': 139852, 'popularity_es': 4312, 'popularity_fr': 3326, 'popularity_it': 1586, 'popularity_pt': 1286}
3
Carolina Schmidt
{'popularity_en': 31, 'popularity_es': 132, 'popularity_fr': 25, 'popularity_it': 0, 'popularity_pt': 0}
4
Donald Trump
{'popularity_en': 51092, 'popularity_es': 4651, 'popularity_fr': 3796, 'popularity_it': 1922, 'popularity_pt': 1871}
5
Jair Bolsonaro
{'popularity_en': 4080, 'popularity_es': 1304, 'popularity_fr': 797, 'popularity_it': 490, 'popularity_pt': 5397}
6
Donald Trump
{'popularity_en': 617453, 'popularity_es': 9681, 'popularity_fr': 7807, 'popularity_it': 4653, 'popularity_pt': 2666}
7
Emmanuel Macron
{'popularity_en': 6178, 'popularity_es': 1126, 'popularity_fr': 

/home/mvernier/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


49
Felipe Larraín
{'popularity_en': 2, 'popularity_es': 89, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
50
Xi Jinping
{'popularity_en': 6601, 'popularity_es': 545, 'popularity_fr': 623, 'popularity_it': 532, 'popularity_pt': 218}
51
Raúl Cordero
{'popularity_en': 3, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
52
Michelle Bachelet
{'popularity_en': 680, 'popularity_es': 888, 'popularity_fr': 160, 'popularity_it': 91, 'popularity_pt': 177}
53
Vladimir Putin
{'popularity_en': 16116, 'popularity_es': 16, 'popularity_fr': 3, 'popularity_it': 1373, 'popularity_pt': 942}
54
Mauricio Macri
{'popularity_en': 1197, 'popularity_es': 2636, 'popularity_fr': 154, 'popularity_it': 142, 'popularity_pt': 249}
55
Xi Jinping
{'popularity_en': 5706, 'popularity_es': 503, 'popularity_fr': 480, 'popularity_it': 294, 'popularity_pt': 160}
56
Obras Públicas
{'popularity_en': 0, 'popularity_es': 3, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 1}
5

/home/mvernier/.local/lib/python3.8/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/mvernier/.local/lib/python3.8/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Catalina Pérez no está en Wikipedia.
103
Cecilia Morel
{'popularity_en': 57, 'popularity_es': 184, 'popularity_fr': 13, 'popularity_it': 5, 'popularity_pt': 4}
104
Eduardo Frei
{'popularity_en': 2, 'popularity_es': 2, 'popularity_fr': 2, 'popularity_it': 1, 'popularity_pt': 3}
105
Estefanía González
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
106
Juan Carlos Jobet
{'popularity_en': 0, 'popularity_es': 80, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
107
Justin Trudeau
{'popularity_en': 8403, 'popularity_es': 607, 'popularity_fr': 911, 'popularity_it': 198, 'popularity_pt': 192}
108
Pete Buttigieg
{'popularity_en': 20843, 'popularity_es': 134, 'popularity_fr': 220, 'popularity_it': 80, 'popularity_pt': 46}
109
Andrés Chadwick
{'popularity_en': 77, 'popularity_es': 509, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 3}
110
Fernando Barros
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popular

...

Consuelo Rosauro Meseguer no está en Wikipedia.
2461
Consuelo Vidal Consuelo Vidal
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
2462
Consuelo Vidal 
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
2463
Consultado Juan Pablo Ramaccioti
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
Consultado Juan Pablo Ramaccioti no está en Wikipedia.
2464
Cristian Johnson
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
2465
Cristina Cornejo
{'popularity_en': 2, 'popularity_es': 4, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
2466
Cristina Dorador
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
2467
Cristina Dorador Universidad de Antofagasta
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, '

David M. Kennedy Experto no está en Wikipedia.
2525
David Pérez
{'popularity_en': 2, 'popularity_es': 2, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
David Pérez no está en Wikipedia.
2526
David Quammen
{'popularity_en': 61, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
2527
David Scott
{'popularity_en': 529, 'popularity_es': 26, 'popularity_fr': 36, 'popularity_it': 27, 'popularity_pt': 11}
2528
De Alberto Hurtado
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
2529
De Gaulle
{'popularity_en': 89, 'popularity_es': 13, 'popularity_fr': 29, 'popularity_it': 8, 'popularity_pt': 2}
2530
De L'Herbe
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
2531
De Niro
{'popularity_en': 15, 'popularity_es': 0, 'popularity_fr': 8, 'popularity_it': 4, 'popularity_pt': 0}
2532
De Stijl
{'popularity_en': 593, 'popularity_es': 145, 'popularity_fr': 96, 

2589
Edgar Mitchell
{'popularity_en': 425, 'popularity_es': 26, 'popularity_fr': 27, 'popularity_it': 19, 'popularity_pt': 9}
2590
Edith Obstchatko
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
Edith Obstchatko no está en Wikipedia.
2591
Eduard Soler
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
2592
Eduardo Cayul Puel
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
Eduardo Cayul Puel no está en Wikipedia.
2593
Eduardo Durán
{'popularity_en': 0, 'popularity_es': 16, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
2594
Eduardo Frei Montalva
{'popularity_en': 81, 'popularity_es': 321, 'popularity_fr': 16, 'popularity_it': 14, 'popularity_pt': 9}
2595
Eduardo VII
{'popularity_en': 0, 'popularity_es': 23, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 35}
2596
Edward Hill
{'popularity_en': 5, 'popularity_es'

2652
FURKA.- Un
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
2653
Fabiana Alves
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
Fabiana Alves no está en Wikipedia.
2654
Fabián Jaksic
{'popularity_en': 0, 'popularity_es': 5, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
2655
Fabián Yaksic
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
Fabián Yaksic no está en Wikipedia.
2656
Felipe Alpízar
{'popularity_en': 0, 'popularity_es': 0, 'popularity_fr': 0, 'popularity_it': 0, 'popularity_pt': 0}
Felipe Alpízar no está en Wikipedia.
2657
Felipe Cabello



KeyboardInterrupt



In [38]:
df_persons_by_year

,Unnamed: 0,year,name,freq,gender,popularity_en,popularity_es,popularity_fr,popularity_it,popularity_pt,wikipedia,birth,profession
0,0,2019,Sebastián Piñera,371,M,951.0,2052.0,172.0,157.0,266.0,Miguel Juan Sebastián Piñera Echenique OMCh CY...,1949,businessman and politician
1,1,2019,Greta Thunberg,292,F,22508.0,5219.0,4404.0,5597.0,1768.0,Greta Tintin Eleonora Ernman Thunberg (Estocol...,2003,activista medioambiental sueca
2,2,2019,Donald Trump,226,M,139852.0,4312.0,3326.0,1586.0,1286.0,"Donald John Trump (born June 14, 1946) is an A...",1946,media personality
3,3,2019,Carolina Schmidt,198,F,31.0,132.0,25.0,0.0,0.0,María Carolina Schmidt Zaldívar (born 9 Septem...,1967,businesswoman
4,4,2018,Donald Trump,155,M,51092.0,4651.0,3796.0,1922.0,1871.0,"Donald John Trump (born June 14, 1946) is an A...",1946,media personality
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10214,10214,2021,Yuval Harary,1,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10215,10215,2021,la Niña,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10216,10216,2021,Álvaro Ortiz,1,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10217,10217,2021,Álvaro Quintanilla,1,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
df_persons_by_year.to_csv("chile_persons_cambioclimatico_final.csv")

In [6]:
df_persons_by_year = pd.read_csv("chile_persons_cambioclimatico_final.csv")
df_persons_by_year

,Unnamed: 0,Unnamed: 0.1,year,name,freq,gender,popularity_en,popularity_es,popularity_fr,popularity_it,popularity_pt,wikipedia,birth,profession
0,0,0,2019,Sebastián Piñera,371,M,951.0,2052.0,172.0,157.0,266.0,Miguel Juan Sebastián Piñera Echenique OMCh CY...,1949,businessman and politician
1,1,1,2019,Greta Thunberg,292,F,22508.0,5219.0,4404.0,5597.0,1768.0,Greta Tintin Eleonora Ernman Thunberg (Estocol...,2003,activista medioambiental sueca
2,2,2,2019,Donald Trump,226,M,139852.0,4312.0,3326.0,1586.0,1286.0,"Donald John Trump (born June 14, 1946) is an A...",1946,media personality
3,3,3,2019,Carolina Schmidt,198,F,31.0,132.0,25.0,0.0,0.0,María Carolina Schmidt Zaldívar (born 9 Septem...,1967,businesswoman
4,4,4,2018,Donald Trump,155,M,51092.0,4651.0,3796.0,1922.0,1871.0,"Donald John Trump (born June 14, 1946) is an A...",1946,media personality
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10214,10214,10214,2021,Yuval Harary,1,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10215,10215,10215,2021,la Niña,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10216,10216,10216,2021,Álvaro Ortiz,1,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10217,10217,10217,2021,Álvaro Quintanilla,1,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
pd.set_option('display.max_rows', 500)

query="""SELECT birth, gender, count(*) as nb_persons
FROM df_persons_by_year WHERE year = 2018 GROUP BY birth, gender ORDER BY birth ASC, gender;"""
result=sqldf(query)
result

,birth,gender,nb_persons
0,None,None,188
1,None,F,236
2,None,M,762
3,None,N,6
4,(1046,F,3
5,(1943,M,1
6,(1943 – 24 November 2020,F,1
7,(Ecuador,None,1
8,(listen),F,1
9,11 November 1945,M,1


In [2]:
import pandas as pd

In [17]:
pd.set_option('display.max_rows', 1300)

query="""SELECT profession, count(*) as nb_persons
FROM df_persons_by_year GROUP BY profession ORDER BY nb_persons DESC;"""
result=sqldf(query)
result

,profession,nb_persons
0,None,7562
1,politician,246
2,lawyer,119
3,actor,62
4,economist,44
5,diplomat,41
6,footballer,29
7,abogado,25
8,writer,23
9,political scientist,20


In [11]:
query="""SELECT name, profession 
FROM df_persons_by_year WHERE profession = "actor" AND year = 2020;"""
result=sqldf(query)
result

,name,profession
0,Javier Bardem,actor
1,Javiera Moreno,actor
2,Lorena Rivera,actor
3,Marco de,actor
4,Margarita Ducci,actor
5,Mariano Fernández,actor
6,Martin Sheen,actor
7,Sean Connery,actor


In [15]:
query="""SELECT name, profession, year 
FROM df_persons_by_year WHERE name = "Leonardo DiCaprio";"""
result=sqldf(query)
result

,name,profession,year
0,Leonardo DiCaprio,actor and film producer,2019
1,Leonardo DiCaprio,actor and film producer,2020
2,Leonardo DiCaprio,actor and film producer,2018
